In [1]:
import numpy as np
import pandas as pd
import string
from itertools import chain

## Functions:

In [2]:
def get_init_node(n):
    """   
    Randomly assigning initial conditions (1 or 0) for each node.
    
    Args:
        n : int
            Number of nodes in the gene regulatory network.
      
    Returns:
        n_init : np.array
            Initial conditions for nodes 1 to n.
    """
    n_init = np.random.choice(a=[1,0], size=(1, n))
    n_init_vals = np.reshape(n_init,n)
    
    return n_init

In [3]:
def get_conn(n, lesion):
    """
    Setting connectivity.
    
    Args:
        n : int
            Total number of nodes in the network.
        lesion: int
            How many nodes will lose their (both incoming and outgoing) edges. (Type2 lesioning)
      
    Returns:
        new_list: list
            List of connectivity after lesioning for n (indicies of the list) nodes. 
    """
    # target nodes:
    list1= np.arange(n)
    list2= np.arange(n)
    
    for i in range(n): # no self-connections
        while list1[i]==i:
            np.random.shuffle((list1))
        while list2[i]==i:
            np.random.shuffle((list2))
            
    for i in range(n):
        while list1[i]==list2[i]: #don't repeat the same node in connections
            list2[i] = np.random.randint(n)
            
    adj_list = np.column_stack((list1,list2))
    
    ## randomly cut some nodes so that some nodes have only one connection:
    chosenlist=np.array([])
    newlist= list(adj_list) 
    howmanyoneconn= np.random.choice(n)
    
    for i in range(howmanyoneconn):
        chosen= np.random.choice(n)
        while chosen in chosenlist:
            chosen= np.random.choice(n)
        chosenlist = np.append(chosenlist, chosen)
        # print('chosen targets index', chosen)
        nestedlist= list(newlist[chosen])
        pick = np.random.randint(0,2)
        # print('pick which specific index to be deleted',pick)
        nestedlist.remove(nestedlist[pick]) 
        # print('after deletion what is left',nestedlist)
        newlist[chosen] = np.array(nestedlist)

    ## lesion here:
    lesioned = [_ for _ in range(lesion)]
    
    if lesion>= 1:
        for lesions in range(lesion):
            lesionthisgene = np.random.choice(n)
            while lesionthisgene in lesioned:
                lesionthisgene = np.random.choice(n)
            newlist[lesionthisgene] = np.array([lesionthisgene])
            lesioned[lesions] = lesionthisgene
    else:
        pass
        
    # Below is making sure that Type-2 lesioning is consistent, meaning, both incoming and outgoing edges are lesioned.
    for outs in range(n):
        if len(newlist[outs])<2: # Two is the max connections allowed in this simulation
            for ll in range(lesion):
                while lesioned[gene] == newlist[outs] and outs != lesioned[ll]: # and [...] because lesioned gene should not be re-assigned with new incoming edges
                    newlist[outs] = np.array(np.random.choice(n))
        else:
            for ind in range(len(newlist[outs])):
                for ll in range(lesion):
                    while lesioned[ll] == newlist[outs][ind] and outs != lesioned[ll]: # and [...] because lesioned gene should not be re-assigned with new incoming edges
                        newlist[outs][ind] = np.array(np.random.choice(n))
    
    
    return newlist

In [4]:
def get_init_states(n, newlist, n_init):
    """
    Given the connectivity, assign their initial values.
    
    Args:
        n : int
            Total number of nodes in the network.
        newlist: list
            List of np.array (connected pairs).

        n_init : np.array
            Initial conditions for nodes 1 to n.

      
    Returns:
        targs: list of np.array.
            List of initial conditions according to their connectivity of n (indicies of the list) nodes. 
    """
    state_vals = [_ for _ in range(n)]

    for i in range(n):
        if newlist[i].size > 1:
            x,y =  newlist[i]
            next_state = n_init[x], n_init[y]
            next_state = np.asarray(next_state)
        else:
            x = newlist[i]
            next_state= n_init[x]
        state_vals[i] = next_state
    return state_vals

In [5]:
def get_bool(state_vals):
    """
    Generate a random set of Boolean rules.
    
    Args:
        state_vals : list of np.array
            List of initial conditions according to their connectivity of n (indicies of the list) nodes. 

    Returns:
        new_values: list of np.array.
            Updated states of the nodes according to Boolean rules.
        ruleset: list of str.
            The Boolean ruleset for update functions.
    """
    pair_operators = ['a','o','an','on','xor','nan','non','na','no'] # shorthand for bool operators
    single_operators = ['not', 'copy']
    ruleset=[]
    
    for i in range(n):
        
        if state_vals[i].size > 1: 
            operator = np.random.choice(pair_operators,1)
            ruleset += [operator.tolist()]
        else:
            operator = np.random.choice(single_operators,1)
            ruleset += [operator]
    
    return ruleset

In [6]:
def get_updated_states(new_values,ruleset,adj_list):
    """
    Update states of nodes given the ruleset previously defined. 
    
    Args:
        new_values : list of np.array
            Previous state of the nodes.
        ruleset : list of str
            Boolean ruleset.
        adj_list : list of np.array
            Connectivity.

    Returns:
        updated_values: np.array.
            Updated states of the nodes according to Boolean rules.
    """
    prev_values = (new_values.copy()).astype(int)
    updated_values = np.zeros(np.shape(prev_values))

    for i in range(n):
        if adj_list[i].size > 1:
            conn_gene1, conn_gene2 =  adj_list[i]
            x, y = prev_values[conn_gene1], prev_values[conn_gene2]
            
            # next_state = np.asarray(next_state)
        else:
            lonely_gene = adj_list[i]
            # next_state = new_values[lonely_gene]

        rule= ruleset[i]
        if rule==['a']:
            updated_values[i]= np.logical_and(x,y)
            # print('hello this is AND',x,y)
        elif rule==['an']:
            updated_values[i]= np.logical_and(x,np.logical_not(y))
            # print('hello this is AND NOT',x,y)

        elif rule==['o']:
            updated_values[i]= np.logical_or(x,y)
            # print('hello this is OR',x,y)

        elif rule==['on']:
            updated_values[i]= np.logical_or(x,np.logical_not(y))
            # print('hello this is OR NOT',x,y)

        elif rule==['xor']:
            updated_values[i] = np.logical_xor(x,y)
        elif rule==['nan']:
            updated_values[i] = np.logical_and(np.logical_not(x),np.logical_not(y))
        elif rule==['non']:
            updated_values[i] = np.logical_or(np.logical_not(x),np.logical_not(y))
        elif rule==['na']:
            updated_values[i] = np.logical_and(np.logical_not(x),y)
        elif rule==['no']:
            updated_values[i] = np.logical_or(np.logical_not(x),y)

        elif rule==['copy']:
            updated_values[i]= new_values[lonely_gene]
        elif rule==['not']:
            updated_values[i]= np.logical_not(new_values[lonely_gene])


    updated_values = updated_values.astype(int)
    return updated_values 

In [9]:
def get_pert_nodes(k):
    """
    Pick the nodes to be perturbed.
    
    Args:
        k : int
            Number of nodes to be perturbed.

    Returns:
        perts: list.
            List of perturbed node indicies.
    """
    pertb = [_ for _ in range(n)]
    perts = pertb[0:k]
    return perts

## Simulations start below:

In [10]:
networks = 1000 # How many networks will be generated for each perturbation condition in a given lesoning parameter
n = 10 # the number of genes

experiments = 10
period = 10
timepoints = experiments * period
burnin = experiments #burn-in at the beginning of each experiment throughout the simulation

for k in range(n+1):
    
    for lesion in range(n+1):
        
        for network in range(networks):
        
            node_inits= get_init_node(n)
            genelist = get_conn(n, lesion)
            gene_states = get_init_states(n, genelist, node_inits)
            new_vals, theruleset = get_bool(gene_states)


            for experiment in range(experiments+1):

                node_inits= get_init_node(n) #changes inits for next round of experiments
                perturbedlist = get_pert_nodes(k) # perturbed gene list

                if experiment > 0:
                    makedf = pd.DataFrame(expdata_each)
                    allexpdata = pd.concat([allexpdata, makedf], axis=1)
                elif experiment == 0:
                    allexpdata = pd.DataFrame({'A' : []})

                expdata = np.zeros((period, n)) #initialize to concat later 

                for i in range(period): 

                    expvector = get_updated_states(new_vals,theruleset,genelist)
                    new_vals = expvector # update the states by feeding prev. here

                    for gene in range(k):
                        per = perturbedlist[gene]

                      #weighted coin flip for deciding perturbing/flipping the state of the node(s)
                        coin= ['heads','tails']
                        flip = np.random.choice(coin, p=[0.25, 0.75]) #before 0.1, 0.9
                        if flip=='heads':
                            new_vals[per] = np.logical_not(expvector[per])
                        # print('after flip vector', new_vals)
                        else:
                            pass #not flip


                    # ADD SMALL FLIPPY THING FOR ALL GENES - INCLUDING PERTURBED ONES
                    for luckygene in range(n):
                        coin= ['heads','tails']
                        flip = np.random.choice(coin, p=[0.05, 0.95])
                        
                        if flip=='heads':
                            new_vals[luckygene] = np.logical_not(expvector[luckygene])
                        else:
                            pass


                    expdata[i]= expvector
                    expdata_each= np.asarray(expdata.T.astype(int))

                    #BURN-IN
                    expdata_each = np.delete(expdata_each, 0, 1)


        #AFTER EXPERIMENTS
        expdatafinal = allexpdata # conc one
        genenames = list([f'G{genenum}' for genenum in range(1,n+1)])
        colnameslist = [[f'Exp{expt}Point{t}' for t in range(1,period)] for expt in range(experiments)] #burnin skipping by range(1,period)
        colnameslist = list(chain.from_iterable(colnameslist))
        df = pd.DataFrame(expdatafinal)
        df.index = genenames
        dataset = df.drop(columns=['A'])
        dataset.columns = colnameslist
        datafinal = f'~/Desktop/YOUR_FOLDER_NAME_HERE/Type2/ExpressionData_k{k}_lesionedgenes{lesion}_net{network}.csv'
        dataset.to_csv(datafinal)